## 1. Tải Dữ Liệu từ CSV

In [1]:
!pip install datasets

In [2]:

from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModel
import numpy as np
import torch
from datasets import load_dataset
import torch.nn as nn
import os
from typing import List
from tqdm import tqdm


# os.environ["CUDA_VISIBLE_DEVICES"] = "1" ## Setup CUDA GPU 1



In [3]:
import torch

# Kiểm tra GPU khả dụng
print("CUDA available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("No GPU found.")


CUDA available: True
Number of GPUs: 1
GPU 0: Tesla T4


In [4]:
import os
import torch

def select_gpu():
    """
    Kiểm tra GPU khả dụng và tự động chọn GPU phù hợp.
    """
    if torch.cuda.is_available():
        num_gpus = torch.cuda.device_count()
        print(f"Number of GPUs available: {num_gpus}")

        # Duyệt qua các GPU khả dụng để tìm GPU ít sử dụng nhất
        available_gpus = [torch.cuda.get_device_name(i) for i in range(num_gpus)]
        print("Available GPUs:", available_gpus)

        for i in range(num_gpus):
            try:
                # Đặt GPU
                os.environ["CUDA_VISIBLE_DEVICES"] = str(i)
                device = torch.device(f"cuda:{i}")
                torch.cuda.set_device(device)
                print(f"Using GPU: {torch.cuda.get_device_name(device.index)}")
                return device
            except Exception as e:
                print(f"GPU {i} is not suitable: {e}")

        print("No suitable GPU found. Falling back to CPU.")
        return torch.device("cpu")
    else:
        print("No GPUs available. Using CPU.")
        return torch.device("cpu")

# Tự động chọn GPU hoặc CPU
device = select_gpu()

# Kiểm tra lại thiết bị đang sử dụng
print(f"Final selected device: {device}")


Number of GPUs available: 1
Available GPUs: ['Tesla T4']
Using GPU: Tesla T4
Final selected device: cuda:0


In [5]:

class BERTIntentClassification(nn.Module):


    def __init__(self, model_name="bert-base-uncased", num_classes=10, dropout_rate=0.1, cache_dir = "huggingface"):
        super(BERTIntentClassification, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name, cache_dir = cache_dir)
        # Get BERT hidden size
        hidden_size = self.bert.config.hidden_size
        self.ffnn = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, num_classes)
        )


    def freeze_bert(self):
        for param in self.bert.parameters():
            param.requires_grad = False


    def get_pooling(self, hidden_state, attention_mask):
        """
        Get mean pooled representation from BERT hidden states
        Args:
            hidden_state: BERT output containing hidden states
        Returns:
            pooled_output: Mean pooled representation of the sequence
        """
        # Get last hidden state
        last_hidden_state = hidden_state.last_hidden_state  # Shape: [batch_size, seq_len, hidden_size]

        if attention_mask is not None:
            # Expand attention mask to match hidden state dimensions
            attention_mask = attention_mask.unsqueeze(-1)  # [batch_size, seq_len, 1]

            # Mask out padding tokens
            masked_hidden = last_hidden_state * attention_mask

            # Calculate mean (sum / number of actual tokens)
            sum_hidden = torch.sum(masked_hidden, dim=1)  # [batch_size, hidden_size]
            count_tokens = torch.sum(attention_mask, dim=1)  # [batch_size, 1]
            pooled_output = sum_hidden / count_tokens
        else:
            # If no attention mask, simply take mean of all tokens
            pooled_output = torch.mean(last_hidden_state, dim=1)

        return pooled_output


    def forward(self, input_ids, attention_mask, **kwargs):
        """
        Forward pass of the model
        Args:
            input_ids: Input token IDs
            attention_mask: Attention mask for padding
        Returns:
            logits: Raw logits for each class
        """
        # Get BERT hidden states
        hidden_state = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )

        # Get pooled representation
        hidden_state_pooling = self.get_pooling(hidden_state=hidden_state, attention_mask=attention_mask)

        # Pass through FFNN classifier
        logits = self.ffnn(hidden_state_pooling)

        return logits


In [6]:
class TrainerCustom(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.

        Subclass and override for custom behavior.
        """
        if "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None

        # Sử dụng nn.CrossEntropyLoss() thay vì nn.CrossEntropy
        cross_entropy_loss = nn.CrossEntropyLoss()

        # Chạy mô hình và nhận đầu ra (logits)
        outputs = model(**inputs)

        # Đảm bảo lấy logits từ outputs (mô hình trả về tuple, lấy phần tử đầu tiên là logits)
        logits = outputs

        # Tính toán loss
        loss = cross_entropy_loss(logits, labels)

        # Trả về loss và outputs nếu cần
        return (loss, outputs) if return_outputs else loss


# 1. Load Dataset and with Dataloader


**Cách kết hợp question và answer:**

1. **Ghép nối trực tiếp:** Bạn có thể kết hợp câu hỏi và câu trả lời thành một chuỗi duy nhất, sử dụng một ký tự đặc biệt hoặc dấu phân cách để tách biệt chúng. Ví dụ:

   ```python
   combined_text = question + " [SEP] " + answer
   ```

   Trong đó, `[SEP]` là một token đặc biệt thường được sử dụng trong các mô hình như BERT để phân tách các đoạn văn bản khác nhau.

2. **Sử dụng token đặc biệt:** Một số mô hình hỗ trợ các token đặc biệt để đánh dấu bắt đầu và kết thúc của câu hỏi và câu trả lời. Ví dụ:

   ```python
   combined_text = "[CLS] " + question + " [SEP] " + answer + " [SEP]"
   ```

   - `[CLS]`: Token đánh dấu bắt đầu chuỗi (thường dùng trong BERT).
   - `[SEP]`: Token phân tách giữa các phần của chuỗi.



Ví dụ thực tế về chuỗi chuẩn:

Một câu/đoạn duy nhất:
```
[CLS] This is the first sentence. [SEP]
```
Hai câu/đoạn (ví dụ: câu hỏi và trả lời):
```
[CLS] What is your name? [SEP] My name is John. [SEP]
```
Nhiều câu/đoạn (3 đoạn):
```
[CLS] Question 1 [SEP] Answer 1 [SEP] Extra information [SEP]
```

```
                          input_ids  intent
0  [CLS] Cậu có muốn tiếp tục không? [SEP]  silence
1                          [CLS] [SEP]  silence

```

###### xỬ LÝ VẤN ĐỀ BỊ THỪA: CLS, SEP CỦA CÁC VERSION TRƯỚC + BERT SẼ HOẠT ĐỘNG TỐT VỚI CHỮ THƯỜNG: lowercase và - question: {question}. answer: {}
- dùng lowercase với BERT mà


Việc thêm `[CLS]` và `[SEP]` theo cách thủ công như bạn thực hiện là **không cần thiết** và có thể **không tương thích** với cách mà tokenizer của BERT được thiết kế hoạt động. Cụ thể:

---

### **1. Hoạt động mặc định của BERT Tokenizer**
- **`[CLS]`**: Là token đặc biệt được thêm tự động vào đầu mỗi câu bởi tokenizer. BERT sử dụng token này để biểu diễn toàn bộ ngữ cảnh của câu.
- **`[SEP]`**: Là token đặc biệt được sử dụng để phân tách các câu khi có nhiều đoạn văn trong một input (e.g., `text` và `text_pair`).

Khi bạn sử dụng `tokenizer` với `text` và `text_pair`, tokenizer **tự động thêm `[CLS]` và `[SEP]`** vào đầu và cuối các câu, nên bạn không cần phải thêm thủ công.

---

### **2. Những vấn đề khi thêm `[CLS]` và `[SEP]` thủ công**
- **Redundant Tokens**: Bạn đang thêm `[CLS]` và `[SEP]` bằng tay, nhưng sau đó tokenizer lại thêm **lần nữa**, dẫn đến chuỗi input bị thừa các token đặc biệt. Ví dụ:

  ```python
  input_text = "[CLS] Cậu có muốn học thêm về các cụm từ khác không? [SEP] Không, tớ không muốn học thêm đâu. [SEP]"
  tokenizer(input_text)
  # Tokenizer sẽ thêm [CLS] và [SEP] lần nữa -> Dẫn đến lỗi hoặc input không chính xác.
  ```

- **Không tuân theo pretraining**: BERT được huấn luyện trước với cơ chế tự động thêm `[CLS]` và `[SEP]`. Nếu bạn thêm bằng tay, mô hình có thể xử lý input không chính xác vì không phù hợp với cách mà mô hình đã được huấn luyện trước đó.

---


1. **Loại Bỏ `[CLS]` và `[SEP]`:**
   - Tokenizer của BERT sẽ tự động thêm `[CLS]` ở đầu chuỗi và `[SEP]` giữa các đoạn văn bản (và ở cuối nếu cần).
   - Điều này giúp tránh dư thừa token.

2. **Chuẩn Hóa Chuỗi:**
   - Chuyển toàn bộ chuỗi về **lowercase** để nhất quán với tokenizer (đặc biệt với BERT mặc định sử dụng lowercase).

3. **Định Dạng Chuỗi:**
   - Sử dụng định dạng chuẩn:
     ```plaintext
     question: {nội dung câu hỏi}. answer: {nội dung câu trả lời}.
     ```


In [50]:
# import pandas as pd
# from datasets import Dataset

# def combine_with_special_tokens(row, text_columns, cls_token="[CLS]", sep_token="[SEP]"):
#     """
#     Thêm các token đặc biệt vào chuỗi kết hợp từ các cột văn bản.

#     Args:
#         row (pd.Series): Dòng dữ liệu từ DataFrame.
#         text_columns (list): Danh sách các cột văn bản cần kết hợp.
#         cls_token (str): Token bắt đầu câu.
#         sep_token (str): Token phân cách.

#     Returns:
#         str: Chuỗi văn bản đã thêm token đặc biệt.
#     """
#     tokens = [cls_token]  # Thêm [CLS] đầu tiên

#     # Thêm nội dung từ các cột văn bản
#     for col in text_columns:
#         if pd.notna(row[col]) and row[col].strip():  # Kiểm tra không rỗng
#             tokens.append(row[col].strip())
#             tokens.append(sep_token)  # Thêm [SEP] sau mỗi đoạn

#     # Nếu không có nội dung nào được thêm, chỉ giữ lại [CLS] và [SEP]
#     if len(tokens) == 1:
#         tokens.append(sep_token)

#     return " ".join(tokens)

# def load_xlsx_dataset(xlsx_path, text_columns, label_column, cls_token="[CLS]", sep_token="[SEP]"):
#     """
#     Tải dataset từ file Excel (.xlsx) và xử lý dữ liệu.

#     Args:
#         xlsx_path (str): Đường dẫn đến file .xlsx.
#         text_columns (list): Danh sách các cột cần ghép để tạo văn bản đầu vào.
#         label_column (str): Tên cột chứa nhãn.
#         cls_token (str): Token bắt đầu câu.
#         sep_token (str): Token phân cách.

#     Returns:
#         Dataset: Tập dữ liệu đã xử lý.
#     """
#     # Đọc file Excel bằng pandas
#     df = pd.read_excel(xlsx_path)

#     # Kiểm tra các cột cần thiết
#     for col in text_columns + [label_column]:
#         if col not in df.columns:
#             raise ValueError(f"Missing required column: {col}")

#     # Ghép các cột text lại thành một chuỗi duy nhất với token đặc biệt
#     df["input_ids"] = df.apply(lambda row: combine_with_special_tokens(row, text_columns, cls_token, sep_token), axis=1)

#     # Đổi tên cột nhãn
#     df = df.rename(columns={label_column: "label"})

#     # Chuyển đổi DataFrame thành Dataset
#     dataset = Dataset.from_pandas(df[["input_ids", "label"]])

#     return dataset

# # Sử dụng hàm
# xlsx_path = "/content/processed_data_example_v4_15000Data.xlsx"  # Đường dẫn file Excel
# text_columns = ["robot", "user_answer"]  # Các cột cần ghép
# label_column = "user_intent"  # Cột chứa nhãn

# # Tải dataset từ Excel
# dataset = load_xlsx_dataset(xlsx_path, text_columns, label_column)

# # Kiểm tra dữ liệu
# print(dataset)

# # Lấy 10 mẫu đầu tiên
# sample_dataset = dataset.select(range(20))
# print(sample_dataset)

# # In thử một hàng
# print("First row in dataset:")
# print(sample_dataset[0])
# print(sample_dataset[11])


Dataset({
    features: ['input_ids', 'label'],
    num_rows: 15694
})
Dataset({
    features: ['input_ids', 'label'],
    num_rows: 20
})
First row in dataset:
{'input_ids': '[CLS] Được rồi, bây giờ chúng ta sẽ chơi một trò chơi! Hãy kể tên nhiều từ thuộc cùng 1 chủ đề nhé. Chủ đề lần này là hành động bắt đầu bằng từ "eat food". Tớ ví dụ nhé, "eat pizza", đến lượt cậu nhé [SEP] Tớ ăn cơm. [SEP]', 'label': 'intent_positive'}
{'input_ids': '[CLS] Cậu có muốn chơi tiếp không? [SEP]', 'label': 'silence'}


In [51]:
import pandas as pd
from datasets import Dataset

def combine_with_format(row, question_col, answer_col):
    """
    Kết hợp dữ liệu từ cột câu hỏi và câu trả lời theo định dạng chuẩn.

    Args:
        row (pd.Series): Dòng dữ liệu từ DataFrame.
        question_col (str): Tên cột chứa câu hỏi.
        answer_col (str): Tên cột chứa câu trả lời.

    Returns:
        str: Chuỗi định dạng chuẩn "question: {câu hỏi}. answer: {câu trả lời}."
    """
    question = row[question_col].strip().lower() if pd.notna(row[question_col]) else ""
    answer = row[answer_col].strip().lower() if pd.notna(row[answer_col]) else ""
    return f"question: {question}. answer: {answer}."

def load_xlsx_dataset_v3(xlsx_path, question_col, answer_col, label_col):
    """
    Tải và xử lý dữ liệu từ file Excel, chuyển đổi định dạng đầu vào.

    Args:
        xlsx_path (str): Đường dẫn đến file Excel.
        question_col (str): Tên cột chứa câu hỏi.
        answer_col (str): Tên cột chứa câu trả lời.
        label_col (str): Tên cột chứa nhãn.

    Returns:
        Dataset: Dữ liệu đã xử lý.
    """
    # Đọc file Excel
    df = pd.read_excel(xlsx_path)

    # Kiểm tra các cột
    for col in [question_col, answer_col, label_col]:
        if col not in df.columns:
            raise ValueError(f"Missing required column: {col}")

    # Kết hợp văn bản
    df["input_ids"] = df.apply(lambda row: combine_with_format(row, question_col, answer_col), axis=1)

    # Giữ nguyên nhãn không ánh xạ
    df = df.rename(columns={label_col: "label"})

    # Chuyển đổi DataFrame thành Dataset
    dataset = Dataset.from_pandas(df[["input_ids", "label"]])

    return dataset

# Sử dụng hàm
xlsx_path = "/content/processed_data_example_v4_15000Data.xlsx"  # File dữ liệu
question_col = "robot"  # Cột câu hỏi
answer_col = "user_answer"  # Cột câu trả lời
label_col = "user_intent"  # Cột nhãn

# Tải dataset từ Excel
dataset = load_xlsx_dataset_v3(xlsx_path, question_col, answer_col, label_col)

# Kiểm tra dữ liệu
print(dataset)

# Lấy 10 mẫu đầu tiên
sample_dataset = dataset.select(range(10))
print(sample_dataset)

# In thử một hàng
print("First row in dataset:")
print(sample_dataset[0])


Dataset({
    features: ['input_ids', 'label'],
    num_rows: 15694
})
Dataset({
    features: ['input_ids', 'label'],
    num_rows: 10
})
First row in dataset:
{'input_ids': 'question: được rồi, bây giờ chúng ta sẽ chơi một trò chơi! hãy kể tên nhiều từ thuộc cùng 1 chủ đề nhé. chủ đề lần này là hành động bắt đầu bằng từ "eat food". tớ ví dụ nhé, "eat pizza", đến lượt cậu nhé. answer: tớ ăn cơm..', 'label': 'intent_positive'}


In [52]:
def check_invalid_samples(dataset):
    invalid_samples = []
    for idx, sample in enumerate(dataset):
        if not isinstance(sample["input_ids"], str) or sample["input_ids"].strip() == "":
            invalid_samples.append((idx, sample))
    return invalid_samples

# Kiểm tra dữ liệu không hợp lệ
invalid_samples = check_invalid_samples(dataset)
print("\n===== Invalid Samples =====")
print(invalid_samples)



===== Invalid Samples =====
[]


In [53]:
# Tự động phát hiện nhãn và tạo ánh xạ nhãn
def create_label_mapping(dataset_list):
    """
    Tự động phát hiện tất cả các nhãn từ danh sách dataset và ánh xạ chúng thành số nguyên.
    """
    all_labels = set()
    for dataset in dataset_list:
        all_labels.update(dataset["label"])  # Tập hợp tất cả các nhãn từ dataset

    label_to_int = {label: idx for idx, label in enumerate(sorted(all_labels))}
    print(f"Ánh xạ nhãn: {label_to_int}")
    return label_to_int

# Hàm chuyển đổi nhãn
def preprocess_labels(example, label_to_int):
    example["label"] = label_to_int.get(example["label"], -1)  # Gán -1 cho nhãn không hợp lệ
    return example

# Tạo ánh xạ nhãn
label_mapping = create_label_mapping([dataset])

# Áp dụng chuyển đổi nhãn
dataset = dataset.map(lambda example: preprocess_labels(example, label_mapping))

# Kiểm tra kết quả
print(dataset)

# Truy cập mẫu cụ thể
sample_dataset = dataset.select(range(10))  # Lấy 10 mẫu đầu tiên
print(sample_dataset)

# In thử 1 hàng trong sample_dataset
print("First row in sample_dataset:")
print(sample_dataset[1])

Ánh xạ nhãn: {'intent_fallback': 0, 'intent_learn_more': 1, 'intent_negative': 2, 'intent_neutral': 3, 'intent_positive': 4, 'silence': 5}


Map:   0%|          | 0/15694 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'label'],
    num_rows: 15694
})
Dataset({
    features: ['input_ids', 'label'],
    num_rows: 10
})
First row in sample_dataset:
{'input_ids': 'question: tốt lắm! cậu có thể kể thêm từ nào khác không?. answer: tớ ăn bánh mì..', 'label': 4}


In [54]:
# def split_dataset(dataset, test_size=0.2, seed=42):
#     """
#     Chia dataset thành tập train và test.

#     Args:
#         dataset (Dataset): Tập dữ liệu đầy đủ.
#         test_size (float): Tỷ lệ dữ liệu test (0.0 - 1.0).
#         seed (int): Seed để chia dữ liệu ngẫu nhiên.

#     Returns:
#         tuple: (train_dataset, test_dataset) - Tập train và test.
#     """
#     if not (0.0 < test_size < 1.0):
#         raise ValueError("test_size phải nằm trong khoảng (0.0, 1.0)")
#     if len(dataset) < 2:
#         raise ValueError("Dataset phải có ít nhất 2 mẫu để chia.")

#     train_test_split = dataset.train_test_split(test_size=test_size, seed=seed)
#     print(f"Chia dataset: {len(train_test_split['train'])} mẫu train, {len(train_test_split['test'])} mẫu test")
#     return train_test_split["train"], train_test_split["test"]

# # Chia dataset
# train_dataset, test_dataset = split_dataset(dataset, test_size=0.3)

# # Kiểm tra dữ liệu
# print("Train dataset:", train_dataset)
# print("Test dataset:", test_dataset)

# # Truy cập mẫu cụ thể
# sample_train_dataset = train_dataset.select(range(8))  # Lấy 10 mẫu đầu tiên từ train
# sample_test_dataset = test_dataset.select(range(5))    # Lấy 10 mẫu đầu tiên từ test

# print("Sample train dataset:", sample_train_dataset)
# print("Sample test dataset:", sample_test_dataset)

In [55]:
def split_dataset(dataset, test_size=0.2, valid_size=0.2, seed=42):
    """
    Chia dataset thành tập train, valid, và test.

    Args:
        dataset (Dataset): Tập dữ liệu đầy đủ.
        test_size (float): Tỷ lệ dữ liệu test (0.0 - 1.0).
        valid_size (float): Tỷ lệ dữ liệu valid so với tập train ban đầu (0.0 - 1.0).
        seed (int): Seed để chia dữ liệu ngẫu nhiên.

    Returns:
        tuple: (train_dataset, valid_dataset, test_dataset) - Tập train, valid, và test.
    """
    if not (0.0 < test_size < 1.0):
        raise ValueError("test_size phải nằm trong khoảng (0.0, 1.0)")
    if not (0.0 < valid_size < 1.0):
        raise ValueError("valid_size phải nằm trong khoảng (0.0, 1.0)")
    if len(dataset) < 3:
        raise ValueError("Dataset phải có ít nhất 3 mẫu để chia.")

    # Chia tập train và test
    train_test_split = dataset.train_test_split(test_size=test_size, seed=seed, shuffle=False)
    train_dataset = train_test_split["train"]
    test_dataset = train_test_split["test"]

    # Chia tập train thành train và valid
    valid_split = train_dataset.train_test_split(test_size=valid_size, seed=seed)
    train_dataset = valid_split["train"]
    valid_dataset = valid_split["test"]

    print(f"Chia dataset: {len(train_dataset)} mẫu train, {len(valid_dataset)} mẫu valid, {len(test_dataset)} mẫu test")
    return train_dataset, valid_dataset, test_dataset


# Chia dataset
train_dataset, valid_dataset, test_dataset = split_dataset(dataset, test_size=0.2, valid_size=0.2)

# Kiểm tra dữ liệu
print("Train dataset:", train_dataset)
print("Valid dataset:", valid_dataset)
print("Test dataset:", test_dataset)

# Truy cập mẫu cụ thể
sample_train_dataset = train_dataset.select(range(8))  # Lấy 8 mẫu đầu tiên từ train
sample_valid_dataset = valid_dataset.select(range(5))  # Lấy 5 mẫu đầu tiên từ valid
sample_test_dataset = test_dataset.select(range(5))    # Lấy 5 mẫu đầu tiên từ test

print("Sample train dataset:", sample_train_dataset)
print("Sample valid dataset:", sample_valid_dataset)
print("Sample test dataset:", sample_test_dataset)


Chia dataset: 10044 mẫu train, 2511 mẫu valid, 3139 mẫu test
Train dataset: Dataset({
    features: ['input_ids', 'label'],
    num_rows: 10044
})
Valid dataset: Dataset({
    features: ['input_ids', 'label'],
    num_rows: 2511
})
Test dataset: Dataset({
    features: ['input_ids', 'label'],
    num_rows: 3139
})
Sample train dataset: Dataset({
    features: ['input_ids', 'label'],
    num_rows: 8
})
Sample valid dataset: Dataset({
    features: ['input_ids', 'label'],
    num_rows: 5
})
Sample test dataset: Dataset({
    features: ['input_ids', 'label'],
    num_rows: 5
})


In [56]:
for i in range(5):
    print(sample_test_dataset[i])

{'input_ids': "question: cậu có muốn học thêm về các cụm từ khác không? maybe something like 'run fast'?. answer: không, tớ không muốn học thêm đâu..", 'label': 2}
{'input_ids': "question: cậu có hiểu cách sử dụng 'jump now' không? do you feel confident using it?. answer: tớ không biết, có thể tớ cần thêm thời gian..", 'label': 3}
{'input_ids': "question: cậu có thể nói lại câu 'jump now' không? can you repeat it?. answer: tớ không chắc lắm, nhưng tớ sẽ thử..", 'label': 3}
{'input_ids': "question: cậu có muốn tìm hiểu thêm về các động từ hành động không? like 'jump'?. answer: có, tớ muốn biết thêm về các động từ khác..", 'label': 1}
{'input_ids': "question: cậu có muốn học cách kết hợp 'jump now' với các câu khác không? for example, 'jump now and have fun!'. answer: ừ, tớ muốn học thêm về cách kết hợp..", 'label': 1}


# 2. Tokenizer

In [57]:
# Calculate the number of unique labels
print(label_mapping)
number_label = len(label_mapping)
print("Number of unique labels:", number_label)

{'intent_fallback': 0, 'intent_learn_more': 1, 'intent_negative': 2, 'intent_neutral': 3, 'intent_positive': 4, 'silence': 5}
Number of unique labels: 6


In [58]:
# Bước 2: Chuẩn bị tokenizer và token hóa dữ liệu
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir = "huggingface")
model = BERTIntentClassification(
    model_name=model_name,
    num_classes=6
)
model.freeze_bert() # Froze Layer BERT
max_seq_length = 512


def collate_fn(features):
    inputs = []
    labels = []
    for element in features:
        inputs.append(element.get("input_ids"))
        labels.append(element.get("label"))

    labels = torch.tensor(labels, dtype=torch.long)

    token_inputs = tokenizer(
        inputs,                           # Chuỗi đầu vào (sau khi ghép nối từ `robot` và `user_answer`).
        add_special_tokens=True,          # Tự động thêm các token đặc biệt như [CLS], [SEP] vào chuỗi đầu vào.
        truncation=True,                  # Cắt bớt chuỗi nếu độ dài vượt quá `max_length`.
        padding=True,                     # Thêm padding (với token [PAD]) để đảm bảo tất cả các chuỗi có độ dài bằng nhau.
        max_length=max_seq_length,        # Độ dài tối đa của chuỗi sau khi token hóa (ví dụ: 512).
        return_overflowing_tokens=False,  # Không trả về các đoạn token vượt quá `max_length`.
        return_length=False,              # Không trả về độ dài của từng chuỗi token đã tạo ra.
        return_tensors="pt",              # Trả về đầu ra dưới dạng tensor của PyTorch (định dạng này cần thiết để sử dụng với mô hình).
    )

    token_inputs.update({
        "labels": labels,
    })
    return token_inputs


# 3. Train Model

## 3.1 Log Wandb

In [59]:
!pip install --upgrade wandb

In [60]:
!pip install python-dotenv

In [61]:
from dotenv import load_dotenv
import os

# Load biến môi trường từ file .env
load_dotenv()

# Lấy key từ biến môi trường
wandb_api_key = os.getenv("WANDB_API_KEY")
print(wandb_api_key[:5])

c8767


In [62]:
import wandb
import os

# Lấy API key từ biến môi trường và đăng nhập
wandb.login(key=os.getenv("WANDB_API_KEY"))


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

Cách thiết lập thông qua TrainingArguments
Khi sử dụng Trainer, bạn có thể đặt tên dự án trực tiếp trong TrainingArguments bằng cách sử dụng tham số report_to và run_name. Tuy nhiên, để đặt project, bạn cần khởi tạo một phiên wandb trước hoặc truyền cấu hình này thông qua wandb.init().

Điều chỉnh TrainingArguments:
```python
training_args = TrainingArguments(
    output_dir="./results_",          # Thư mục lưu kết quả
    eval_strategy="epoch",           # Đánh giá sau mỗi epoch
    learning_rate=2e-4,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",            # Thư mục lưu log
    logging_strategy="steps",        # Log theo steps
    logging_steps=10,                # Log sau mỗi 10 bước
    save_strategy="epoch",           # Lưu checkpoint sau mỗi epoch
    save_total_limit=3,              # Lưu tối đa 3 checkpoint
    report_to="wandb",               # Báo cáo log tới wandb
    run_name="bert_run_1"            # Tên phiên chạy trên wandb
)
```

## 3.2 Train

### Ver 1.2.3

Thui, ko lưu local nữa, lưu tất trên wandb đi.
- Với best model: lưu lên wandb khi loss giảm và đã sau 10 epochs  
(Lưu Best Model ngay khi eval_loss giảm ở local, sau 10 epochs thì đồng bộ cái best lên wandb, sau đó xoá các file best ở local).
Chỉ đồng bộ lên WandB mỗi 10 epochs.)
- Với last model: lưu lên wandb sau mỗi 10 epochs. (lưu local trước -> đồng bộ lên wandb sẽ xoá file local)
+, Trong quá trình lưu thì việc training vẫn diễn ra Parallel

đều lưu đầy đủ toàn bộ tham số để có thể train thêm từ cả ở best model và last model

In [63]:
from concurrent.futures import ThreadPoolExecutor
import wandb
import os
import shutil
import time

class TrainerCustom(Trainer):
    def __init__(self, *args, save_every_n_epochs=10, **kwargs):
        super().__init__(*args, **kwargs)
        if torch.cuda.is_available():
            print(f"Trainer is running on GPU: {torch.cuda.get_device_name(torch.cuda.current_device())}")
        else:
            print("Trainer is running on CPU.")

        self.best_eval_loss = float("inf")  # Giá trị loss tốt nhất ban đầu
        self.save_every_n_epochs = save_every_n_epochs  # Tần suất lưu lên WandB
        self.best_model_info = {"epoch": None, "loss": None}
        self.last_saved_epoch = 0  # Epoch cuối cùng đã lưu Best Model và Last Model
        self.executor = ThreadPoolExecutor(max_workers=3)  # Cho phép tối đa 2 luồng song song

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.

        Subclass and override for custom behavior.
        """

        # # Kiểm tra thiết bị của mô hình và dữ liệu
        # print("Model device:", next(model.parameters()).device)
        # print("Input device:", inputs["input_ids"].device)
        if "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None

        # Sử dụng nn.CrossEntropyLoss() thay vì nn.CrossEntropy
        cross_entropy_loss = nn.CrossEntropyLoss()

        # Chạy mô hình và nhận đầu ra (logits)
        outputs = model(**inputs)

        # Đảm bảo lấy logits từ outputs (mô hình trả về tuple, lấy phần tử đầu tiên là logits)
        logits = outputs

        if labels is None:
            print("Labels are None during compute_loss.")
        if logits is None:
            print("Logits are None during compute_loss.")

        # Tính toán loss
        loss = cross_entropy_loss(logits, labels)

        # Trả về loss và outputs nếu cần
        return (loss, outputs) if return_outputs else loss

    def async_save_model(self, model_dir, artifact_name, metadata=None):
        """
        Lưu mô hình vào local và đồng bộ lên WandB trong luồng song song.
        """
        def save():
            start_time = time.time()
            try:
                # Xóa tất cả các thư mục tmp_best_model_ trước đó
                for folder in os.listdir("."):
                    if folder.startswith("tmp_best_model_epoch_") and folder != model_dir:
                        shutil.rmtree(folder, ignore_errors=True)
                        print(f"Removed old temporary directory: {folder}")

                # Lưu mô hình vào thư mục tạm
                self.save_model(model_dir)

                # Đồng bộ lên WandB
                artifact = wandb.Artifact(artifact_name, type="model")
                artifact.add_dir(model_dir)
                if metadata:
                    artifact.metadata = metadata
                wandb.log_artifact(artifact)
            except Exception as e:
                print(f"Error during saving or syncing model {artifact_name}: {e}")
            finally:
                # Xóa thư mục tạm hiện tại sau khi đồng bộ
                try:
                    shutil.rmtree(model_dir, ignore_errors=True)
                    print(f"Successfully removed temporary directory: {model_dir}")
                except Exception as e:
                    print(f"Error removing temporary directory {model_dir}: {e}")

            elapsed_time = time.time() - start_time
            print(f"Model saved and uploaded to WandB: {artifact_name} in {elapsed_time:.2f} seconds")

        self.executor.submit(save)




    def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix: str = "eval"):
        metrics = super().evaluate(eval_dataset, ignore_keys, metric_key_prefix)
        eval_loss = metrics.get("eval_loss")

        # Cập nhật Best Model nếu eval_loss giảm
        # Lưu Best Model ngay khi eval_loss giảm (local).
        # Đồng bộ wandb ngay

        if eval_loss is not None and eval_loss < self.best_eval_loss:
            print(f"New best eval_loss: {eval_loss}")
            self.best_eval_loss = eval_loss
            self.best_model_info = {"epoch": self.state.epoch, "loss": eval_loss}

            # Log thông tin Best Model lên WandB
            wandb.log({
                "best_eval_loss": self.best_eval_loss,
                "best_model_epoch": self.best_model_info.get("epoch", -1)
            })

            # Lưu Best Model vào thư mục tạm (local)
            best_model_dir = f"./tmp_best_model_epoch_{int(self.state.epoch)}"
            self.save_model(best_model_dir)

            # Đồng bộ lên WandB ngay
            artifact_name = f"best_model_epoch_{int(self.state.epoch)}"
            self.async_save_model(best_model_dir, artifact_name, self.best_model_info)

        return metrics

    # def save_last_model(self):
    #     """
    #     Lưu Last Model lên WandB sau mỗi N epochs.
    #     """
    #     if int(self.state.epoch) % self.save_every_n_epochs == 0 and int(self.state.epoch) != self.last_saved_epoch:
    #         print(f"Saving Last Model at epoch {self.state.epoch} to WandB...")
    #         last_model_dir = f"./tmp_last_model_epoch_{int(self.state.epoch)}"
    #         artifact_name = f"last_model_epoch_{int(self.state.epoch)}"
    #         self.async_save_model(last_model_dir, artifact_name)

    #         # Log thông tin Last Model lên WandB
    #         wandb.log({
    #             "last_model_epoch": self.state.epoch
    #         })

    #         # Cập nhật epoch cuối cùng đã lưu
    #         self.last_saved_epoch = int(self.state.epoch)

    def save_last_model(self):
        """
        Lưu Last Model (bao gồm trạng thái optimizer, scheduler) lên WandB sau mỗi N epochs.
        """
        if int(self.state.epoch) % self.save_every_n_epochs == 0 and int(self.state.epoch) != self.last_saved_epoch:
            print(f"Saving Last Model at epoch {self.state.epoch} to WandB...")

            # Thư mục tạm lưu checkpoint
            last_model_dir = f"./tmp_last_model_epoch_{int(self.state.epoch)}"
            os.makedirs(last_model_dir, exist_ok=True)

            # Lưu đầy đủ trạng thái mô hình (checkpoint)
            self.save_model(last_model_dir)

            # Đường dẫn tệp trainer_state.json
            trainer_state_path = os.path.join(last_model_dir, "trainer_state.json")
            self.state.save_to_json(trainer_state_path)  # Lưu trạng thái trainer

            # Đồng bộ checkpoint lên WandB
            artifact_name = f"last_model_epoch_{int(self.state.epoch)}"
            metadata = {
                "epoch": int(self.state.epoch),
                "last_eval_loss": self.state.best_metric if hasattr(self.state, "best_metric") else "N/A",
            }
            self.async_save_model(last_model_dir, artifact_name, metadata)

            # Cập nhật epoch cuối cùng đã lưu
            self.last_saved_epoch = int(self.state.epoch)


    def train(self, *args, **kwargs):
        result = super().train(*args, **kwargs)

        # Sau mỗi epoch, lưu Last Model lên WandB
        self.save_last_model()
        # Chờ tất cả các luồng lưu hoàn thành trước khi kết thúc
        self.executor.shutdown(wait=True)

        return result


# Bước 6: Cài đặt tham số huấn luyện
training_args = TrainingArguments(
    output_dir="./result__s",          # Thư mục lưu kết quả
    eval_strategy="epoch",    # Đánh giá sau mỗi epoch
    learning_rate=2e-4,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=1,  # Ghi logs mỗi 500 bước huấn luyện
    save_strategy="no",          # Lưu trọng số sau mỗi epoch
    save_total_limit=3,
    label_names = ["labels"],
    report_to="wandb",
    run_name="bert_run_3"
)


import wandb

# Khởi tạo wandb
wandb.init(
    project="bertIntentClassification",  # Tên dự án
    name="bert_10000Data_1epoch",                     # Tên phiên chạy
    config={"gpu": torch.cuda.get_device_name(torch.cuda.current_device()) if torch.cuda.is_available() else "CPU"}
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
if torch.cuda.is_available():
    print(f"Trainer is running on GPU: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("Trainer is running on CPU.")

trainer = TrainerCustom(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=collate_fn,
    save_every_n_epochs=3  # Lưu Best Model và Last Model mỗi 10 epochs
)

trainer.train()


wandb.finish()


Trainer is running on GPU: Tesla T4
Trainer is running on GPU: Tesla T4


Epoch,Training Loss,Validation Loss
1,0.827600,0.795858
2,0.708100,0.702948
3,0.673800,0.597507
4,0.768500,0.566722
5,0.681100,0.536791
6,0.609000,0.529781
7,0.369300,0.513642
8,0.612300,0.500463
9,0.452800,0.482030
10,0.588400,0.475604


New best eval_loss: 0.7958577275276184
Removed old temporary directory: tmp_best_model_epoch_1


wandb: Adding directory to artifact (./tmp_best_model_epoch_1)... Done. 3.3s


Successfully removed temporary directory: ./tmp_best_model_epoch_1
Model saved and uploaded to WandB: best_model_epoch_1 in 10.23 seconds
New best eval_loss: 0.7029483318328857
Removed old temporary directory: tmp_best_model_epoch_2


...

wandb: Adding directory to artifact (./tmp_best_model_epoch_9)... Done. 6.9s


Successfully removed temporary directory: ./tmp_best_model_epoch_9
Model saved and uploaded to WandB: best_model_epoch_9 in 19.04 seconds
New best eval_loss: 0.47560402750968933
Removed old temporary directory: tmp_best_model_epoch_10


wandb: Adding directory to artifact (./tmp_best_model_epoch_10)... Done. 2.5s


Successfully removed temporary directory: ./tmp_best_model_epoch_10
Model saved and uploaded to WandB: best_model_epoch_10 in 8.46 seconds


best_eval_loss,█▆▄▃▂▂▂▂▁▁
best_model_epoch,▁▂▃▃▄▅▆▆▇█
eval/loss,█▆▄▃▂▂▂▂▁▁
eval/runtime,▁█▄▄▄▄▄▅▄▅
eval/samples_per_second,█▁▅▅▄▅▄▄▅▄
eval/steps_per_second,█▁▅▅▄▅▄▄▄▄
test/loss,▁▁▁
test/runtime,█▁█
test/samples_per_second,▁█▁
test/steps_per_second,▁█▁
train/epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇███


### Test

### Test

In [47]:
# # Khắc phục lỗi không đồng nhất số lượng nhãn và dự đoán
# test_results = trainer.predict(test_dataset)

# # Cắt nhãn và dự đoán theo độ dài nhỏ nhất
# predictions = test_results.predictions.argmax(axis=-1)
# labels = test_results.label_ids

# min_length = min(len(labels), len(predictions))
# labels = labels[:min_length]
# predictions = predictions[:min_length]

# # Báo cáo hiệu suất
# from sklearn.metrics import classification_report
# report = classification_report(
#     labels,
#     predictions,
#     target_names=["intent_fallback", "intent_learn_more", "intent_negative", "intent_neutral", "intent_positive", "silence"]
# )
# print("\nClassification Report:\n", report)



Classification Report:
                    precision    recall  f1-score   support

  intent_fallback       0.15      0.11      0.12       481
intent_learn_more       0.14      0.14      0.14       509
  intent_negative       0.17      0.20      0.18       544
   intent_neutral       0.15      0.17      0.16       536
  intent_positive       0.16      0.17      0.17       551
          silence       0.16      0.15      0.15       493

         accuracy                           0.16      3114
        macro avg       0.16      0.16      0.15      3114
     weighted avg       0.16      0.16      0.16      3114



In [ ]:
import os
import json
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
import wandb
import pandas as pd
import os
import time



# 1. Tải mô hình từ artifact trên WandB
run = wandb.init(project="bert-intent-classification")  # Tên dự án trong WandB
artifact = run.use_artifact('doanngoccuong_nh/bertIntentClassification/best_model_epoch_9:v1', type='model')
artifact_dir = artifact.download()
print("Files in artifact_dir:", os.listdir(artifact_dir))

# Đường dẫn tệp cấu hình
config_path = os.path.join(artifact_dir, "config.json")

# Kiểm tra và cập nhật tệp config.json
config = {
    "model_type": "bert",
    "hidden_size": 768,
    "num_attention_heads": 12,
    "num_hidden_layers": 12,
    "vocab_size": 30522
}
with open(config_path, "w") as f:
    json.dump(config, f, indent=4)
print(f"Config.json updated at {config_path}")


# 2. Create config.json if not available
config = AutoConfig.from_pretrained("bert-base-uncased")

# Tải tokenizer từ mô hình gốc
original_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Lưu các tệp cần thiết vào artifact_dir
original_tokenizer.save_pretrained(artifact_dir)

print(f"Tokenizer files saved to {artifact_dir}")

# 4. Tải mô hình đã lưu và tokenizer
model_path = artifact_dir  # Đường dẫn đến mô hình đã tải
tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModelForSequenceClassification.from_pretrained(model_path)



# Khởi tạo mô hình trống
from safetensors.torch import load_file
model = BERTIntentClassification(model_name="bert-base-uncased", num_classes=6)
weights_path = os.path.join(artifact_dir, "model.safetensors") # Đường dẫn đến tệp `model.safetensors`
state_dict = load_file(weights_path) # Tải trọng số vào mô hình
model.load_state_dict(state_dict)
model.eval()


# Chuyển mô hình sang chế độ đánh giá
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(f"Model loaded and running on device: {device}")




# Bước 2: Chuẩn bị tokenizer và token hóa dữ liệu
max_seq_length = 512

# def preprocess_input(question, answer, tokenizer, max_seq_length):
#     """
#     Tiền xử lý dữ liệu đầu vào bằng cách ghép nối câu hỏi và câu trả lời với các token đặc biệt.
#     """
#     input_text = f"[CLS] {question.strip()} [SEP] {answer.strip()} [SEP]"
#     inputs = tokenizer(
#         input_text,
#         return_tensors="pt",
#         truncation=True,
#         padding=True,
#         max_length=max_seq_length
#     )
#     return inputs

def preprocess_input(question, answer, tokenizer, max_seq_length):
    """
    Tiền xử lý dữ liệu đầu vào bằng cách kết hợp câu hỏi và câu trả lời theo định dạng chuẩn.
    """
    # Kết hợp câu hỏi và câu trả lời theo định dạng chuẩn
    input_text = f"question: {question.strip().lower()}. answer: {answer.strip().lower()}."

    # Tokenize dữ liệu đầu vào
    # inputs = tokenizer(
    #     input_text,
    #     return_tensors="pt",  # Trả về tensor PyTorch
    #     truncation=True,      # Cắt chuỗi nếu vượt quá độ dài tối đa
    #     padding="max_length", # Thêm padding để đạt độ dài tối đa
    #     max_length=max_seq_length  # Độ dài tối đa
    # )

    # return inputs

    # Tokenize dữ liệu đầu vào
    token_inputs = tokenizer(
        inputs,                           # Chuỗi đầu vào (sau khi ghép nối từ `robot` và `user_answer`).
        add_special_tokens=True,          # Tự động thêm các token đặc biệt như [CLS], [SEP] vào chuỗi đầu vào.
        truncation=True,                  # Cắt bớt chuỗi nếu độ dài vượt quá `max_length`.
        padding=True,                     # Thêm padding (với token [PAD]) để đảm bảo tất cả các chuỗi có độ dài bằng nhau.
        max_length=max_seq_length,        # Độ dài tối đa của chuỗi sau khi token hóa (ví dụ: 512).
        return_overflowing_tokens=False,  # Không trả về các đoạn token vượt quá `max_length`.
        return_length=False,              # Không trả về độ dài của từng chuỗi token đã tạo ra.
        return_tensors="pt",              # Trả về đầu ra dưới dạng tensor của PyTorch (định dạng này cần thiết để sử dụng với mô hình).
    )

    return token_inputs

# 6. Khởi tạo biến lưu kết quả
results = []
correct_predictions = 0

def map_label(pred_class, label_mapping):
    return label_mapping.get(pred_class, f"Unknown (Class ID: {pred_class})")

# Cập nhật label_mapping từ thông tin huấn luyện
label_mapping = {
    0: "intent_fallback",
    1: "intent_learn_more",
    2: "intent_negative",
    3: "intent_neutral",
    4: "intent_positive",
    5: "silence"
}

# 7. Thực hiện inference trên từng dòng dữ liệu
import os
import pandas as pd
import shutil
import torch

def process_and_update_file(input_file, output_file, model, tokenizer, label_mapping, max_seq_length, device, num_rows=None):
    """
    Processes an input Excel file, performs inference, and updates the file with predicted results.

    Args:
        input_file (str): Path to the input Excel file.
        output_file (str): Path to the output Excel file.
        model: The trained model for inference.
        tokenizer: Tokenizer for preprocessing.
        label_mapping (dict): Mapping from class index to label.
        max_seq_length (int): Maximum sequence length for the tokenizer.
        device: PyTorch device (e.g., 'cpu' or 'cuda').
        num_rows (int, optional): Number of rows to process. Default is None (process all rows).
    """
    # Sao chép file gốc nếu file output chưa tồn tại
    if not os.path.exists(output_file):
        shutil.copy(input_file, output_file)
        print(f"File copied from {input_file} to {output_file}")

    # Đọc dữ liệu từ file output
    data = pd.read_excel(output_file)

    # Giới hạn số dòng nếu cần
    if num_rows is not None:
        data = data.head(num_rows)
        print(f"Processing only the first {num_rows} rows.")

    # Xử lý inference và thêm cột mới
    results = []
    correct_predictions = 0



    for idx, row in data.iterrows():
        question = row["robot"]
        answer = row["user_answer"] if not pd.isna(row["user_answer"]) else ""
        true_intent = row["user_intent"]

        # Tiền xử lý đầu vào
        inputs = preprocess_input(question, answer, tokenizer, max_seq_length)
        print("input processed:", inputs)
        inputs = {key: value.to(device) for key, value in inputs.items()}

        # Thực hiện dự đoán
        start_time = time.time()
        with torch.no_grad():
            logits = model(**inputs)  # Custom model directly returns logits
            predicted_class = torch.argmax(logits, dim=1).item()
            predicted_label = map_label(predicted_class, label_mapping)
        end_time = time.time()

        # Calculate response time
        response_time = end_time - start_time

        # Kiểm tra đúng sai
        is_correct = (predicted_label == true_intent)
        if is_correct:
            correct_predictions += 1

        # Lưu kết quả
        results.append({
            "predicted_intent": predicted_label,
            "is_correct": is_correct,
            "model_response_time": response_time
        })

        print(f"Question: {question}")
        print(f"Answer: {answer}")
        print(f"Inputs: {inputs}")
        print(f"Logits: {logits}")
        print(f"Predicted class: {predicted_class}")
        print(f"Predicted label: {predicted_label}")

    # Tạo DataFrame từ kết quả
    results_df = pd.DataFrame(results)

    # Thêm cột vào DataFrame ban đầu
    data["predicted_intent"] = results_df["predicted_intent"]
    data["is_correct"] = results_df["is_correct"]
    data["model_response_time"] = results_df["model_response_time"]

    # Ghi kết quả trở lại file Excel
    with pd.ExcelWriter(output_file, engine="openpyxl", mode="w") as writer:
        data.to_excel(writer, index=False)

    # Tính accuracy
    accuracy = correct_predictions / len(data)
    print(f"Accuracy: {accuracy:.2%}")
    print(f"Evaluation results saved to {output_file}")

# Định nghĩa các tham số cần thiết
input_file = "/content/test1_processed_TEST_500to1000Phrase.xlsx"
output_file = "evaluation_results.xlsx"
num_rows = 5000  # Số lượng dòng muốn đánh giá
# model = ...  # Model đã huấn luyện
# tokenizer = ...  # Tokenizer tương ứng
# label_mapping = {0: "intent_A", 1: "intent_B", 2: "intent_C"}  # Mapping nhãn
# max_seq_length = 128  # Độ dài tối đa
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Gọi hàm để xử lý và cập nhật file
# Định nghĩa các tham số cần thiết


# Gọi hàm để xử lý và giới hạn số dòng
process_and_update_file(input_file, output_file, model, tokenizer, label_mapping, max_seq_length, device, num_rows=num_rows)




- 3500 Test (Ko Shuffle): 75.95%


2. Đánh giá trên bộ 600 câu được tạo bởi 50 cụm mới hoàn toàn:
- Với BERT cũ: Để bình thường và CLS, SEP bị double => Accuracy: 68.16% (10 epochs).
Hôm qua (best epoch 90/100 epochs) đánh giá trên data mới toanh này chỉ đạt 46.67%

- Khi Chuyển sang: lowercase và question: {question}. answer: {answer} độ chính xác lên : 69.65% (10 epochs).
